In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from itertools import groupby


df = pd.read_csv("EURUSD_RATES")

# Determine candle color
def get_candle_color(open, close):
    if close > open:
        return 'green'
    elif close < open:
        return 'red'
    else:
        return 'neutral'

df['color'] = df.apply(lambda row: get_candle_color(row['open'], row['close']), axis=1)

# Initialize merged dataframe
merged_candles = []

# Function to merge a sequence of candles
def merge_sequence(sequence, color):
    if color == 'red':
        new_open = max(sequence['open'])
        new_close = min(sequence['close'])
        new_high = max(sequence['high'])
        new_low = min(sequence['low'])
    elif color == 'green':
        new_open = min(sequence['open'])
        new_close = max(sequence['close'])
        new_high = max(sequence['high'])
        new_low = min(sequence['low'])
    else:  # neutral
        new_open = min(sequence['open'])
        new_close = max(sequence['close'])
        new_high = max(sequence['high'])
        new_low = min(sequence['low'])

    first_datetime = sequence['time'].iloc[0]  # Keep the datetime of the first candle
    merged_candles.append({
        'time': first_datetime,  # Save the datetime of the first candle in the sequence
        'open': new_open,
        'close': new_close,
        'high': new_high,
        'low': new_low,
        'color': color
    })

# Iterate over the dataframe and merge consecutive candles of the same color
current_sequence = []
current_color = None

for idx, row in df.iterrows():
    if row['color'] == current_color:
        current_sequence.append(row)
    else:
        if current_sequence:
            # Merge the previous sequence before starting a new one
            merge_sequence(pd.DataFrame(current_sequence), current_color)
        current_sequence = [row]
        current_color = row['color']

# Merge the last sequence
if current_sequence:
    merge_sequence(pd.DataFrame(current_sequence), current_color)

# Convert merged candles to a new dataframe
merged_df = pd.DataFrame(merged_candles)
merged_df


,time,open,close,high,low,color
0,1678172400,1.06903,1.06764,1.06905,1.06761,red
1,1678179600,1.06763,1.06856,1.06945,1.06747,green
2,1678183200,1.06856,1.06568,1.06863,1.06555,red
3,1678190400,1.06569,1.06605,1.06664,1.06542,green
4,1678194000,1.06605,1.06567,1.06688,1.06553,red
...,...,...,...,...,...,...
5135,1728648000,1.09495,1.09309,1.09511,1.09273,red
5136,1728658800,1.09309,1.09429,1.09506,1.09260,green
5137,1728669600,1.09428,1.09413,1.09524,1.09410,red
5138,1728673200,1.09413,1.09448,1.09475,1.09403,green
